# Comparisons between two portfolios - example

This script illustrates the **azapy** tools to carry out a comparative analysis between 2 portfolios.

We start by importing **azapy** and other useful packages.

In [17]:
import pandas as pd

import sys
sys.path.append("..")
import azapy as az

### Load historical market data for 2 different portfolios.

The portfolio components are listed in `symb1` and `symb2`, respectively.

In [18]:
symb1 = ['GLD', 'TLT', 'XLV', 'VGT', 'PSJ']
symb2 = ['PFG', 'QQQ', 'SPY', 'XLF', 'ONEQ']

sdate = pd.to_datetime("2012-01-01")
edate = pd.to_datetime("today")

mktdir = "./MkTdata"

mktdata1 = az.readMkT(symb1, dstart = sdate, dend = edate, dir=mktdir, force=False) 
mktdata2 = az.readMkT(symb2, dstart = sdate, dend = edate, dir=mktdir, force=False) 

Read GLD form ./MkTdata
Read TLT form ./MkTdata
Read XLV form ./MkTdata
Read VGT form ./MkTdata
Read PSJ form ./MkTdata
Read PFG form ./MkTdata
Read QQQ form ./MkTdata
Read SPY form ./MkTdata
Read XLF form ./MkTdata
Read ONEQ form ./MkTdata


### Check the historical market data quality for fist portfolio

We use the **azapy** function summary_MktData. It returns:

- `symbol` the symbols included in `mktdata1`. These are the portfolio components. 
- `begin` is the beginning of the historical time-series for each symbol.
- `end` is the end date of the historical time series (the most recent date).
- `length` is history length in number of business days.
- `na_total` is the total number of missing values, `nan`.
- `na_b` is the total number of missing records at the beginning of the time-series (in number of business days).
- `na_e` is the total number of missing records at the end of the time-series (in number of business days).
- `cont` is the total number of missing records (at the beginning, end and in the middle of the time-series). 

We are looking `na-total`, `na_b`, `na_e` and `count` to be 0. Missing data could be a serious problem since in general gaps should be filled before proceeding further. 

*For more information see the documentation for summary_MkTData*

In [19]:
az.summary_MkTData(mktdata1)

,symbol,begin,end,length,na_total,na_b,na_e,cont
0,GLD,2012-01-03,2021-08-12,2419,0,0,0,0
1,PSJ,2012-01-03,2021-08-12,2419,0,0,0,0
2,TLT,2012-01-03,2021-08-12,2419,0,0,0,0
3,VGT,2012-01-03,2021-08-12,2419,0,0,0,0
4,XLV,2012-01-03,2021-08-12,2419,0,0,0,0


### Check the historical market data quality for second portfolio

Same as before.

In [20]:
az.summary_MkTData(mktdata2)

,symbol,begin,end,length,na_total,na_b,na_e,cont
0,ONEQ,2012-01-03,2021-08-12,2419,0,0,0,0
1,PFG,2012-01-03,2021-08-12,2419,0,0,0,0
2,QQQ,2012-01-03,2021-08-12,2419,0,0,0,0
3,SPY,2012-01-03,2021-08-12,2419,0,0,0,0
4,XLF,2012-01-03,2021-08-12,2419,0,0,0,0


### Set the dispersion measure parameters.

In our case we have chosen a mixture of CVaR's a dispersion measure. Similar workflow holds if other dispersion measure was choosed.

In [21]:
alpha = [0.95, 0.90, 0.85]
coef = [0.2, 0.3, 0.5]

### Setup the first portfolio 

Two steps:

1. Construct the portfolio class. In our case this is `Port_CVaR`.
2. Set the model and trigger the computation of the portfolio time-series.

*For more information see the documentation for `Port_CVaR` class*

In [22]:
pr1 = az.Port_CVaR(mktdata1, pname='Port1')
port1 = pr1.set_model(mu=0., alpha=alpha, coef=coef)

### Setup the second portfolio

Same as above.

In [23]:
pr2 = az.Port_CVaR(mktdata2, pname='Port2')
port2 = pr2.set_model(mu=0., alpha=alpha, coef=coef)

### Setup a `Port_Simple` class to compare the 2 portfolios

The `Port_Simple` class is set from the list of originar portfolio time-series. 

*Note the must call to `set_model` method.*

In [24]:
pp = az.Port_Simple([port1, port2])
_ = pp.set_model()

### Visualized the 2 portfolios time-series.

> Note the use of flag `fancy=True`. It designates the use of *plotly* package for interactive time-series. 

As expected `Port2` is more aggressive than `Port1`

In [25]:
_ = pp.port_view_all(componly=True, fancy=True)

### Compare the performances of the two portfolios

- `RR` is the portfolio rate of returns for the duration of the investment in an annualized form. 
- `DD` is the maximum drawdown rate.
- `Beta` is the ratio `RR/DD`.
- `DD_date` is the date of maximum drawdown.
- `DD_start` is the date when the maximum drawdown event had started.
- `DD_end` is the date when the maximum drawdown event had ended. If it is `Nan` then the drawdown is still in progress (`DD` and `DD_date` are only provisional).

> Note the use of flag `fancy=True` - the rates are reported in percent with 2 decimals.

As expected from the graph, the rate of returns for `Port2` is almost double than for `Port1`. However the maximum drawdown is 50% larger for `Port2` than for `Port1`.  

In [26]:
pp.port_perf(componly=True, fancy=True)

,RR,DD,Beta,DD_date,DD_start,DD_end
symbol,,,,,,
Port2,18.14,-28.56,0.635183,2020-03-16,2020-02-19,2020-06-05
Port1,9.46,-18.30,0.516926,2020-03-20,2020-02-19,2020-06-23


### Compare the portfolios annual returns 

With the exception of 2018, `Port2` has significantly higher returns than `Port1`.
This report can be detailed further by asking for the monthly returns. However in this case it is worthed to investigate separately the performance of the two portfolios.

In [27]:
pp.port_annual_returns(withcomp=True, componly=True, fancy=True)

symbol,Port1,Port2
year,,
2015,-2.17%,-1.71%
2016,1.30%,9.42%
2017,26.06%,32.07%
2018,0.28%,-5.54%
2019,24.15%,30.58%
2020,11.88%,34.96%
2021,-1.31%,16.35%


### First portfolio performance (details)

Note that the portfolio drawdown is smaller than any individual component drawdown. Thus, `Port1` has a good diversification for a moderate average return.

In [28]:
pr1.port_perf(fancy=True)

,RR,DD,Beta,DD_date,DD_start,DD_end
symbol,,,,,,
Port1,9.46,-18.30,0.516926,2020-03-20,2020-02-19,2020-06-23
VGT,23.23,-31.86,0.729150,2020-03-23,2020-02-19,2020-06-09
PSJ,21.86,-30.69,0.712415,2020-03-16,2020-02-19,2020-05-22
XLV,16.80,-28.39,0.591816,2020-03-23,2020-01-22,2020-07-15
TLT,4.65,-21.34,0.218029,2021-03-18,2020-08-04,NaN
GLD,0.53,-42.11,0.012592,2015-12-17,2012-10-04,2020-07-22


### First portfolio drawdown (details)

Subsequent drawdowns are smaller but still above 10%

In [29]:
pr1.port_drawdown(fancy=True)

,DD,Date,Start,End
No,,,,
1,-18.30,2020-03-20,2020-02-19,2020-06-23
2,-15.73,2018-12-24,2018-08-31,2019-02-25
3,-14.62,2015-09-28,2015-08-05,2016-06-02
4,-10.58,2021-03-08,2020-08-06,NaN
5,-9.91,2019-10-22,2019-07-26,2020-01-03


### First portfolio rebalancing weights

Note that recently the portfolio is dominated by GLD. From the beginning of 2020 GLD is over 40%.

In [30]:
pr1.get_weights(fancy=True)

,Droll,Dfix,Dhist,GLD,PSJ,TLT,VGT,XLV
0,2015-06-25,2015-06-24,2012-03-23,0.00,0.00,10.74,0.00,89.26
1,2015-09-25,2015-09-24,2012-06-22,0.00,0.00,13.12,0.00,86.88
2,2015-12-28,2015-12-24,2012-09-24,0.00,35.48,34.84,11.61,18.07
3,2016-03-28,2016-03-24,2012-12-24,0.00,0.39,40.35,21.04,38.22
4,2016-06-27,2016-06-24,2013-03-22,0.00,16.09,42.00,12.00,29.92
5,2016-09-27,2016-09-26,2013-06-26,0.00,22.59,48.18,4.66,24.56
6,2016-12-27,2016-12-23,2013-09-23,0.00,0.00,38.86,61.14,0.00
7,2017-03-28,2017-03-27,2013-12-27,0.00,0.00,36.89,63.11,0.00
8,2017-06-27,2017-06-26,2014-03-26,0.00,0.00,36.02,63.98,0.00
9,2017-09-26,2017-09-25,2014-06-25,0.00,6.08,35.82,58.11,0.00


### Second portfolio performance (details)

Note that the maximum drawdown of `Port2` is the same as for QQQ, with a rate of returns smaller than QQQ. The `Port2` diversification is not better than for QQQ. Overall `Port2` is less attractive than a buy-and-hold position in QQQ.

In [31]:
pr2.port_perf(fancy=True)

,RR,DD,Beta,DD_date,DD_start,DD_end
symbol,,,,,,
Port2,18.14,-28.56,0.635183,2020-03-16,2020-02-19,2020-06-05
QQQ,22.67,-28.56,0.793757,2020-03-16,2020-02-19,2020-06-03
ONEQ,20.93,-30.16,0.694118,2020-03-23,2020-02-19,2020-06-05
SPY,16.13,-33.70,0.478715,2020-03-23,2020-02-19,2020-08-10
XLF,16.39,-42.83,0.382721,2020-03-23,2020-02-14,2021-01-06
PFG,14.50,-64.71,0.224097,2020-03-23,2018-01-26,2021-05-05


### First portfolio rebalancing weights

As expected with very few exceptions `Port2` is technical a single asset portfolio of QQQ.

In our opinion `Port2` is an unattractive investment. If the rate of returns and the drawdown levels are somehow acceptable for an investor, then she is better off by investing directly in a buy-and-hold strategy in QQQ.

In [32]:
pr2.get_weights(fancy=True)

,Droll,Dfix,Dhist,ONEQ,PFG,QQQ,SPY,XLF
0,2015-06-25,2015-06-24,2012-03-23,0.0,0.00,0.00,86.14,13.86
1,2015-09-25,2015-09-24,2012-06-22,0.0,4.22,34.00,0.00,61.78
2,2015-12-28,2015-12-24,2012-09-24,0.0,10.60,89.40,0.00,0.00
3,2016-03-28,2016-03-24,2012-12-24,0.0,0.00,100.00,0.00,0.00
4,2016-06-27,2016-06-24,2013-03-22,0.0,0.00,100.00,0.00,0.00
5,2016-09-27,2016-09-26,2013-06-26,0.0,0.00,100.00,0.00,0.00
6,2016-12-27,2016-12-23,2013-09-23,0.0,0.00,100.00,0.00,0.00
7,2017-03-28,2017-03-27,2013-12-27,0.0,0.00,100.00,0.00,0.00
8,2017-06-27,2017-06-26,2014-03-26,0.0,0.00,100.00,0.00,0.00
9,2017-09-26,2017-09-25,2014-06-25,0.0,0.00,100.00,0.00,0.00
